*Задание 1*

*На 1 миллион показов баннера пришлось 1222 клика. Согласуется ли это с гипотезой о том, что CTR равен 0,1%? Ответ обоснуйте.*

 -------------------
 Нельзя сказать со стопроцентной вероятностью, согласуется гипотеза с реальностью или нет, но мы можем воспользоваться доверительным интервалом и сделать вывод с определённой степенью достоверности.
 Количество "успехов" в серии независимых событий, где вероятность "успеха" постоянна в каждом событии, имеет биноминальное распределение. Чем больше событий - тем ближе биноминальное распределение к нормальному распределению с мат. ожиданием  np и дисперсией npq (p - это вероятность "успеха", q = (1 - p), n - количество событий). Зная это - можем использовать правило трёх сигм.

In [1]:
n = 1000000
p = 0.001
q = 1 - p

mean = p*n
var = n*p*q
sigma = var ** (1 / 2)
three_sig_rule = 1000 + 3 * sigma

if 1222 > three_sig_rule:
    print('Гипотеза про CTR = 0.1% не верна с вероятность более, чем 99.73%')

Гипотеза про CTR = 0.1% не верна с вероятность более, чем 99.73%


*Задание 2*

*В приложении содержится файл “logs_sample.xlsx”, который представляет собой выборку логов открытий писем email-рассылок (адреса пользователей закодированы в столбце cid). Используя любой язык программирования напишите алгоритм, который считает среднее количество открытий писем на пользователя (т. е. на уникальное значение cid) в этом файле. При расчете учитывайте только тех пользователей, которые совершили 2 и более открытий писем. В условии приведена небольшая часть логов. Рассмотрите два случая, при которых ваш алгоритм должен обрабатывать:*

  *- файл логов размером 300 MB*

  *- файл логов размером 300 GB*

---------------------------
Для файла 300 мб:

In [9]:
import pandas as pd

logs = pd.read_excel(r'S:\jupyter\Rostel\logs_sample.xlsx', usecols='A, I')
uniq_id = logs.groupby('cid')
print('Среднее количество открываний писем на пользователя (только те, которые совершили более двух открываний писем): {}'.format(uniq_id.count()[uniq_id.count() >= 2].mean()[0]))

Среднее количество открываний писем на пользователя (только те, которые совершили более двух открываний писем): 2.392523364485981
Wall time: 181 ms


Для файла размером в 30 гб:

- Первый вариант, если важна память. Mergesort лучше показывает себя на больших массивах с точки зрения потребления памяти
- Второй вариант, если важна скорость. Храним все уникальные значения и число их вхождений, но зато не сортируем

In [10]:
import pandas as pd

logs = pd.read_excel(r'S:\jupyter\Rostel\logs_sample.xlsx', usecols='I', squeeze=True).sort_values(kind='mergesort')

prev_id = None
duration = 0
users = 0
opens_total = 0
for cid in logs:
    if cid == prev_id:
        duration += 1
    else:
        if duration >= 2:
            users += 1
            opens_total += duration
        duration = 1
        prev_id = cid

print(opens_total / users)

2.392523364485981
Wall time: 174 ms


In [11]:
from openpyxl import load_workbook
wb = load_workbook(r'S:\jupyter\Rostel\logs_sample.xlsx', read_only=True)
logs = wb.worksheets[0]

all_cids = dict()
for row in logs:
    cid = row[8].value
    if cid not in all_cids:
        all_cids[cid] = 1
    else:
        all_cids[cid] += 1

opens_total = 0
users = 0
for val in all_cids.values():
    if val >= 2:
        opens_total += val
        users += 1

print(opens_total/users)

2.392523364485981
Wall time: 222 ms


*Задание 3*

*Маркетинговый отдел провел две рекламные кампании нового игрового тарифа, размещая баннеры на различных сайтах. CTR одной кампании составил 0,5%, у второй - 0,6%. Как бы вы оценили какую из этих кампаний считать более удачной?*

------------------
Посчитал бы рейтинг конверсии, а если это невозможно - посмотрел бы, сколько пользователь задерживался на сайте после перехода по баннеру.